## Setup Infra

In [ ]:
!ecs-cli --version

ecs-cli version 1.21.0 (bb0b8f0)


In [ ]:
# configure ECS
!ecs-cli configure \
 --cluster udacity-dend-cluster2 \
 --default-launch-type FARGATE \
 --region us-east-1

INFO[0000] Saved ECS CLI cluster configuration default. 


In [ ]:
!ecs-cli up \
--keypair udacity-dend-cluster \
--capability-iam \
--tags project=udacity-dend-cluster2,owner=sparsh

WARN[0004] Enabling container instance tagging because containerInstanceLongArnFormat is enabled for your identity, arn:aws:iam::390354360073:root. If this is not your account default setting, your instances will fail to join your cluster. You can use the PutAccountSettingDefault API to change your account default. 
INFO[0004] Created cluster                               cluster=udacity-dend-cluster2 region=us-east-1
INFO[0008] Waiting for your cluster resources to be created... 
INFO[0009] Cloudformation stack status                   stackStatus=CREATE_IN_PROGRESS
INFO[0072] Cloudformation stack status                   stackStatus=CREATE_IN_PROGRESS
VPC created: vpc-03d8f2846f18b0e1a
Subnet created: subnet-0de67148a6f9bd082
Subnet created: subnet-0ee85c4a81eb26dee
Cluster creation succeeded.


In [ ]:
!aws ec2 authorize-security-group-ingress \
        --group-id sg-0afb868cb6df80e99 \
        --protocol tcp \
        --port 9042 \
        --cidr 0.0.0.0/0

{
    "Return": true,
    "SecurityGroupRules": [
        {
            "SecurityGroupRuleId": "sgr-0508577cb730682f9",
            "GroupId": "sg-0afb868cb6df80e99",
            "GroupOwnerId": "390354360073",
            "IsEgress": false,
            "IpProtocol": "tcp",
            "FromPort": 9042,
            "ToPort": 9042,
            "CidrIpv4": "0.0.0.0/0"
        }
    ]
}


In [ ]:
%%writefile ecs-params.yml
version: 1
task_definition:
  ecs_network_mode: awsvpc
  task_execution_role: ecsTaskExecutionRole
  task_size:
    cpu_limit: 2048
    mem_limit: 8GB
  services:
    cassandra:
      essential: true
run_params:
  network_configuration:
    awsvpc_configuration:
      subnets:
        - subnet-0de67148a6f9bd082
        - subnet-0ee85c4a81eb26dee
      assign_public_ip: ENABLED

Overwriting ecs-params.yml


In [ ]:
!ecs-cli compose \
--project-name udacity-dend-cluster-cassandra \
--file cassandra-docker-compose.yml \
--debug service up  \
--deployment-max-percent 100 \
--deployment-min-healthy-percent 0

DEBU[0000] Parsing the compose yaml...                  
DEBU[0000] Docker Compose version found: 3              
DEBU[0000] Parsing v3 project...                        
DEBU[0000] Parsing the ecs-params yaml...               
DEBU[0000] Parsing the ecs-registry-creds yaml...       
DEBU[0000] Transforming yaml to task definition...      
DEBU[0002] Finding task definition in cache or creating if needed  TaskDefinition="{\n  ContainerDefinitions: [{\n      Command: [],\n      Cpu: 0,\n      DnsSearchDomains: [],\n      DnsServers: [],\n      DockerSecurityOptions: [],\n      EntryPoint: [],\n      Environment: [],\n      Essential: true,\n      ExtraHosts: [],\n      Image: \"cassandra:latest\",\n      Links: [],\n      LinuxParameters: {\n        Capabilities: {\n\n        },\n        Devices: []\n      },\n      MountPoints: [{\n          ContainerPath: \"/var/lib/cassandra\",\n          ReadOnly: false,\n          SourceVolume: \"cassandra_volumne\"\n        }],\n      Name: \"cass

In [ ]:
!ecs-cli ps

Name                                                              State    Ports                          TaskDefinition                    Health
udacity-dend-cluster2/bde5dbcaf140410f976cb53dfe0a76f0/cassandra  RUNNING  18.215.248.188:9042->9042/tcp  udacity-dend-cluster-cassandra:2  UNKNOWN


## Data Modeling

In [ ]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

Creating list of filepaths to process original event csv data files

In [ ]:
filepath = './event_data'
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))

# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [ ]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


Creating a Cluster

In [ ]:
from cassandra.cluster import Cluster

In [ ]:
cluster = Cluster(['18.215.248.188'],port=9042)

# To establish connection and begin executing queries, need a session
session = cluster.connect()

In [ ]:
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS sparkify
        WITH REPLICATION =
        { 'class': 'SimpleStrategy', 'replication_factor': 1 }
    """)
except Exception as error:
    print(error)

In [ ]:
try:
    session.set_keyspace('sparkify')
except Exception as error:
    print(error)

Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

Create queries to ask the following three questions of the data
1. Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [ ]:
try:
    session.execute("""
        CREATE TABLE IF NOT EXISTs songs (
            session_id INT,
            item_in_session INT,
            artist_name TEXT, 
            song_title TEXT, 
            song_length FLOAT, 
            PRIMARY KEY(session_id, item_in_session)
        )
    """)
except Exception as error:
    print(error)  

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs ( session_id, item_in_session, artist_name, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [ ]:
# Do a SELECT to verify that the data have been inserted into each table
query = """
    SELECT artist_name, song_title, song_length FROM songs 
    WHERE session_id = 338 AND item_in_session = 4
"""
try:
    songs_results = session.execute(query)
except Exception as error:
    print(error)

In [ ]:
df = pd.DataFrame(songs_results._current_rows)
print(df)

  artist_name                       song_title  song_length
0   Faithless  Music Matters (Mark Knight Dub)   495.307312


In [ ]:
try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS users (
            user_id INT,
            session_id INT,
            item_in_session INT,
            artist_name TEXT,
            song_title TEXT,
            first_name TEXT,
            last_name TEXT,
            PRIMARY KEY ((user_id, session_id), item_in_session)
        )
    """)
except Exception as error:
    print(error)  

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO users (user_id, session_id, item_in_session, artist_name, song_title, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [ ]:
query = """
    SELECT artist_name, song_title, first_name, last_name FROM users 
    WHERE user_id = 10 AND session_id = 182
"""
try:
    users_results = session.execute(query)
except Exception as error:
    print(error)

In [ ]:
df = pd.DataFrame(users_results._current_rows)
print(df)

         artist_name                                         song_title  \
0   Down To The Bone                                 Keep On Keepin' On   
1       Three Drives                                        Greece 2000   
2  Sebastien Tellier                                          Kilometer   
3      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...   

  first_name last_name  
0     Sylvie      Cruz  
1     Sylvie      Cruz  
2     Sylvie      Cruz  
3     Sylvie      Cruz  


In [ ]:
# User composite key (song_title, user_id) to create unique primary key 
# and match the question for querying specific song by different users
try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS song_users (
            song_title TEXT,
            user_id INT,
            first_name TEXT,
            last_name TEXT,
            PRIMARY KEY (song_title, user_id)
        )
    """)
except Exception as error:
    print(error)  

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_users (song_title, user_id, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [ ]:
query = """
    SELECT first_name, last_name FROM song_users 
    WHERE song_title = 'All Hands Against His Own'
"""
try:
    results = session.execute(query)
except Exception as error:
    print(error)

In [ ]:
df = pd.DataFrame(results._current_rows)
print(df)

   first_name last_name
0  Jacqueline     Lynch
1       Tegan    Levine
2        Sara   Johnson


Drop the tables before closing out the sessions

In [ ]:
try:
    session.execute("DROP TABLE IF EXISTS songs")
    session.execute("DROP TABLE IF EXISTS users")
    session.execute("DROP TABLE IF EXISTS users_songs")
except Exception as error:
    print(error)  

Close the session and cluster connection

In [ ]:
session.shutdown()
cluster.shutdown()